# 🛍️ | Cora-For-Zava: Multi-Agent Pattern

Welcome! In this notebook, you'll create a **multi-agent system** that uses relevant tools to answer customer questions about products.

## 🛒 Our Zava Scenario

**Cora** is a customer service chatbot for **Zava** - a fictitious retailer of home improvement goods for DIY enthusiasts. As Zava retail stores grow, Cora needs to handle more complex customer needs. This notebook shows you how to evolve from a single agent to a multi-agent system, with specialized agents for inventory management and customer service. You'll learn to orchestrate multiple agents working together to provide sophisticated and role-specific assistance.

## 🎯 What You'll Build

Two specialized agents for a hardware store:
- **Product Inventory Agent** - Technical expert for product specs and stock levels
- **Customer Service Agent** - Friendly helper for recommendations and guidance

Both agents will search the same product database but respond differently based on their role.

## 💡 What You'll Learn

- How to connect agents to Azure AI Search
- How to create agents with different personalities
- How to route questions to the right agent
- How to orchestrate multiple agents working together

Ready to build a multi-agent system? Let's get started! 🚀

---

## Step 1: Verify Python Packages

Let's check that the required packages are installed:
- `agent-framework` - Microsoft's agent framework
- `azure-identity` - Azure authentication

In [ ]:
# Verify required packages are installed
import importlib.metadata

try:
    agent_framework_version = importlib.metadata.version('agent-framework')
    azure_identity_version = importlib.metadata.version('azure-identity')
    print("✅ All required packages are installed!")

except importlib.metadata.PackageNotFoundError as e:
    print(f"❌ Missing package: {e}")

## Step 2: Import Libraries

Import the tools we need:
- `ChatAgent` - For interacting with agents
- `CitationAnnotation` - For handling search citations
- `AzureAIAgentClient` - Connects to Azure AI agents
- `AIProjectClient` - Manages Azure AI project resources
- `ConnectionType` - Enum for connection types (like Azure AI Search)
- `AzureCliCredential` - Handles Azure authentication

In [ ]:
import os

from agent_framework import ChatAgent, CitationAnnotation
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity.aio import AzureCliCredential

print("✅ All libraries imported successfully!")

## Step 3: Load Configuration

We need three settings from your environment:
- **Project endpoint** - Your Azure AI Foundry project URL
- **Model name** - The chat model to use (like `gpt-4o`)
- **Search index** - The product database name (`zava-products`)

These were set up in the earlier labs.

In [ ]:
# Get environment variables
project_endpoint = os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT")
model_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT") or os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT")
search_index_name = os.environ.get("AZURE_AI_SEARCH_INDEX_NAME") or os.environ.get("AZURE_SEARCH_INDEX_NAME")

# Verify that the variables are set
if not project_endpoint:
    raise ValueError("❌ AZURE_EXISTING_AIPROJECT_ENDPOINT environment variable is not set!")
if not model_deployment_name:
    raise ValueError("❌ AZURE_OPENAI_DEPLOYMENT environment variable is not set!")
if not search_index_name:
    raise ValueError("❌ AZURE_AI_SEARCH_INDEX_NAME environment variable is not set!")

print("✅ Environment variables loaded successfully!")

## Step 4: Verify Azure Login

Check that you're logged into Azure using the Azure CLI (`az login`).

In [ ]:
# Create credential (will be used throughout the notebook)
credential = AzureCliCredential()

# Verify it works by getting a token
try:
    token = await credential.get_token("https://cognitiveservices.azure.com/.default")
    print("✅ Azure CLI credentials created successfully!")
    print(f"✅ Token retrieved - credential is working!")
except Exception as e:
    print(f"❌ Failed to get token: {e}")
    print("💡 Make sure you've run 'az login' in the terminal first!")
    raise

## Step 5: Connect to Azure AI Project

Create a connection to your Azure AI project and find the Azure AI Search connection that was set up earlier.

In [ ]:
# Create Azure AI Project Client
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

# Get the Azure AI Search connection ID from the project
ai_search_conn_id = ""
async for connection in project_client.connections.list():
    if connection.type == ConnectionType.AZURE_AI_SEARCH:
        ai_search_conn_id = connection.id
        print(f"✅ Found Azure AI Search connection: {connection.name}")
        break

if not ai_search_conn_id:
    raise ValueError("❌ No Azure AI Search connection found in the project!")

print(f"✅ Azure AI Project Client created successfully!")

## Step 6: Define Agent Instructions

Let's write the instructions for our two agents. Think of these as the "personality" and "job description" for each agent.

**Note on Tool Usage:**  
The agents will automatically decide when to use the search tool based on their instructions. To encourage consistent tool usage, we've written the instructions to emphasize searching the product database.

### About the Product Database
The `zava-products` index contains:
- Product names and descriptions
- Prices and stock levels
- Technical specifications
- 1536-dimensional embeddings for smart search

### Product Inventory Agent Instructions

This agent focuses on technical details, specifications, and inventory.

In [ ]:
# Product Inventory Agent Instructions
product_inventory_instructions = """You are a Product Inventory Agent for Zava, a hardware and home improvement company.

Your role:
- Search the product database to provide detailed technical specifications for tools, hardware, and paint products
- Use the search tool to find specific details like: finish types, materials, power specs, prices
- Check and report product availability and stock levels from search results
- When products are found with low stock or are unavailable, search for and suggest alternative products
- Compare products from search results and recommend the best options based on customer needs
- Use technical terminology appropriately

When answering:
- ALWAYS use the search tool to find current product information
- Include specific product names, SKUs, prices, and stock levels from search results
- When stock is low (under 10 units) or unavailable, proactively search for alternatives
- Be professional, precise, and thorough
- Cite specific products found in the search results

Example response structure:
"Based on my search of our product database, I found the following options:
- [Product Name] ([SKU]): [specs] - $[price] - [stock status]
- [Alternative if needed]
Let me know if you need more details on any of these products!"
"""

print("✅ Zava Inventory Agent instructions defined!")
print(f"📝 Instruction length: {len(product_inventory_instructions)} characters")

### Customer Service Agent Instructions

This agent is friendly and helpful, guiding customers to the right products.

In [ ]:
# Customer Service Agent Instructions
customer_service_instructions = """You are a Customer Service Agent for Zava, a hardware and home improvement company.

Your role:
- Help customers with general inquiries, product selection, and personalized recommendations
- Use the search tool to find products that match customer needs
- Provide warm, friendly, and helpful assistance based on actual product data
- Keep your responses concise and informative 
- Always begin with a friendly greeting and end with a follow-up question to continue assisting them

CRITICAL - Response Format (ALWAYS follow this structure):
1. Start with a relevant emoji that matches the topic (🔨 for hammers, 🎨 for paint, 🔧 for tools, etc.)
2. Use the search tool to find relevant products
3. Provide a clear, factual response based on search results in 1 brief sentence,
4. End with a helpful follow-up question to continue assisting them

Example response structure:
"🔨 [Emoji] I found some great options in our catalog! [Search result with brief, informative response]. What type of project are you working on? [Follow-up question]"

When answering:
- ALWAYS use the search tool to find current product information
- ALWAYS use the emoji + search-based response + follow-up question format
- Use friendly, conversational language
- Focus on helping solve the customer's problem based on actual products
- Ask clarifying questions to better understand their needs
- Be empathetic and customer-focused
- Keep responses concise but informative
- Reference specific products from search results
"""

print("✅ Zava Customer Agent instructions defined!")
print(f"📝 Instruction length: {len(customer_service_instructions)} characters")

## Step 7: Create the Agents

Now create both agents with Azure AI Search capabilities. Each agent gets:
- Access to the product search index
- Their own personality (instructions)
- The same search tool configured for semantic search (understanding meaning, not just keywords)

In [ ]:
# Create Product Inventory Agent with Azure AI Search
product_inventory_agent = await project_client.agents.create_agent(
    model=model_deployment_name,
    name="Zava-Inventory-Agent-MAF",
    instructions=product_inventory_instructions,
    tools=[{"type": "azure_ai_search"}],
    tool_resources={
        "azure_ai_search": {
            "indexes": [
                {
                    "index_connection_id": ai_search_conn_id,
                    "index_name": search_index_name,
                    "query_type": "semantic",  
                }
            ]
        }
    },
)

print("✅ Product Inventory Agent created!")
print(f"   Agent ID: {product_inventory_agent.id}")
print(f"   Agent Name: {product_inventory_agent.name}")
print(f"   Tools: azure_ai_search")
print(f"   Search Index: {search_index_name}")

# Create Customer Service Agent with Azure AI Search
customer_service_agent = await project_client.agents.create_agent(
    model=model_deployment_name,
    name="Zava-Customer-Agent-MAF",
    instructions=customer_service_instructions,
    tools=[{"type": "azure_ai_search"}],
    tool_resources={
        "azure_ai_search": {
            "indexes": [
                {
                    "index_connection_id": ai_search_conn_id,
                    "index_name": search_index_name,
                    "query_type": "semantic",  
                }
            ]
        }
    },
)

print("\n✅ Customer Service Agent created!")
print(f"   Agent ID: {customer_service_agent.id}")
print(f"   Agent Name: {customer_service_agent.name}")
print(f"   Tools: azure_ai_search")
print(f"   Search Index: {search_index_name}")

print("\n🎉 Multi-agent team with Azure AI Search is ready!")
print("\n💡 Both agents use the same search index but behave differently based on their instructions!")

## Step 8: Test the Agents

Let's try each agent with the same question to see how they respond differently.

### Test: Product Inventory Agent

In [ ]:
# Test Product Inventory Agent with search
question = "I need to paint my bathroom wall. What paints do you have available?"

print("📦 Product Inventory Agent Response (with search):")
print("=" * 80)
print(f"User: {question}\n")

# Create ChatAgent wrapper for the product inventory agent
chat_client = AzureAIAgentClient(
    agents_client=project_client.agents,
    agent_id=product_inventory_agent.id
)

try:
    async with ChatAgent(chat_client=chat_client) as agent:
        
        # Collect the response and citations
        citations: list[CitationAnnotation] = []
        response_text = ""
        
        async for chunk in agent.run_stream(question):
            if chunk.text:
                response_text += chunk.text
            
            # Collect citations from Azure AI Search responses
            for content in getattr(chunk, "contents", []):
                annotations = getattr(content, "annotations", [])
                if annotations:
                    citations.extend(annotations)
        
        # Display the complete response
        print(f"Agent: {response_text}\n")
        
        # Display citations if any
        if citations:
            print("📚 Citations from Azure AI Search:")
            for i, citation in enumerate(citations, 1):
                print(f"  [{i}] {citation.url}")

finally:
    # Ensure cleanup
    await chat_client.close()

print("=" * 80)
print("\n💡 Notice how the agent searches the product database and provides specific product details!")

### Test: Customer Service Agent

Now ask the same question to the customer service agent.

In [ ]:
# Test Customer Service Agent with search
print("😊 Customer Service Agent Response (with search):")
print("=" * 80)
print(f"User: {question}")

# Create ChatAgent wrapper for the customer service agent
chat_client = AzureAIAgentClient(
    agents_client=project_client.agents,
    agent_id=customer_service_agent.id
)

try:
    async with ChatAgent(chat_client=chat_client) as agent:
        
        # Collect the response and citations
        citations: list[CitationAnnotation] = []
        response_text = ""
        
        async for chunk in agent.run_stream(question):
            if chunk.text:
                response_text += chunk.text
            
            # Collect citations from Azure AI Search responses
            for content in getattr(chunk, "contents", []):
                annotations = getattr(content, "annotations", [])
                if annotations:
                    citations.extend(annotations)
        
        # Display the complete response
        print(f"Agent: {response_text}\n")
        
        # Display citations if any
        if citations:
            print("📚 Citations from Azure AI Search:")
            for i, citation in enumerate(citations, 1):
                print(f"  [{i}] {citation.url}")
finally:
    # Ensure cleanup
    await chat_client.close()

print("=" * 80)
print("\n💡 Notice the different approaches:")
print("   • Product Inventory: Technical search-based details with SKUs and specifications")
print("   • Customer Service: Emoji + friendly search-based recommendations + follow-up question")

## Step 9: Build a Smart Router

Let's create a function that automatically sends questions to the right agent based on keywords in the question.

In [ ]:
async def route_to_agent(question: str):
    """
    Smart router that determines which agent should handle the question based on keywords.
    
    Technical queries (specs, stock, comparisons) → Product Inventory Agent
    General queries (recommendations, help, advice) → Customer Service Agent
    
    Args:
        question: The customer's question
        
    Returns:
        Tuple of (agent_name, response_text, citations)
    """
    # Keywords for technical/inventory queries - specific, detailed information requests
    technical_keywords = [
        # Technical specifications
        "spec", "specification", "specifications", "dimension", "dimensions", 
        "size", "power", "voltage", "amperage", "watt", "rpm", "torque",
        "weight", "length", "width", "height", "capacity",
        
        # Inventory and availability
        "stock", "stock level", "availability", "available", "in stock",
        "out of stock", "quantity", "how many",
        
        # Comparison and alternatives
        "compare", "comparison", "difference", "alternative", "alternatives",
        "versus", "vs", "instead", "replace", "replacement",
        
        # Detailed information
        "technical", "detail", "details", "feature", "features", 
        "model", "model number", "sku", "part number", "price",
        "show me all", "list all", "what's the",
        
        # Product attributes
        "coverage", "finish", "voc", "ion", "battery", "cordless",
        "thread", "gauge", "bristle", "nap", "grit"
    ]
    
    # Keywords for general/service queries - recommendations and help
    service_keywords = [
        # Help and recommendations
        "help", "recommend", "recommendation", "suggest", "suggestion",
        "best", "good", "better", "which one", "what kind",
        "advice", "guide", "should i", "what should",
        
        # Customer needs
        "looking for", "need", "need to", "want", "want to",
        "trying to", "planning to", "going to",
        
        # Project-based
        "project", "use for", "working on", "doing", "build",
        "fix", "repair", "install", "paint", "drill",
        
        # General inquiries
        "can you", "do you have", "is there", "are there",
        "tell me about", "explain", "how", "why", "when"
    ]
    
    question_lower = question.lower()
    
    # Count keyword matches
    technical_score = sum(1 for kw in technical_keywords if kw in question_lower)
    service_score = sum(1 for kw in service_keywords if kw in question_lower)
    
    # Route based on scores (if tied, default to customer service for better experience)
    if technical_score > service_score:
        agent = product_inventory_agent
        agent_name = "Product Inventory Agent"
    else:
        agent = customer_service_agent
        agent_name = "Customer Service Agent"
    
    # Create ChatAgent wrapper and get response
    chat_client = AzureAIAgentClient(
        agents_client=project_client.agents,
        agent_id=agent.id
    )
    
    try:
        async with ChatAgent(chat_client=chat_client) as chat_agent:
            citations: list[CitationAnnotation] = []
            response_text = ""
            
            async for chunk in chat_agent.run_stream(question):
                if chunk.text:
                    response_text += chunk.text
                
                # Collect citations
                for content in getattr(chunk, "contents", []):
                    annotations = getattr(content, "annotations", [])
                    if annotations:
                        citations.extend(annotations)
            
            return (agent_name, response_text, citations)
    finally:
        await chat_client.close()

In [ ]:
# Test the router with different types of questions
test_questions = [
    "I'm looking for interior paint for my bedroom. Can you help?",
    "I need to fix my deck. What materials do you recommend?",
    "What's the difference between deck stains and eggshell paints?",
]

print("🔀 Testing Multi-Agent Router with Azure AI Search\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n### Test {i} ###")
    print(f"❓ Question: {question}\n")
    
    agent_name, response, citations = await route_to_agent(question)
    
    print(f"🤖 Routed to: {agent_name}")
    print(f"💬 Response:\n{response}")
    
    if citations:
        print(f"\n📚 Citations:")
        for j, citation in enumerate(citations, 1):
            print(f"  [{j}] {citation.url}")
    
    print("=" * 80)

## Step 10: Clean Up

When you're done, delete the agents to avoid using resources. Let's define a cleanup function.

In [ ]:
# Define cleanup function
async def cleanup_agents():
    """
    Clean up all agents and close connections.
    Call this when you're done testing to avoid using resources.
    """
    print("🧹 Cleaning up resources...")
    
    # Delete both agents
    await project_client.agents.delete_agent(product_inventory_agent.id)
    print(f"✅ Deleted Product Inventory Agent (ID: {product_inventory_agent.id})")
    
    await project_client.agents.delete_agent(customer_service_agent.id)
    print(f"✅ Deleted Customer Service Agent (ID: {customer_service_agent.id})")
    
    # Close connections
    await project_client.close()
    await credential.close()
    
    print("✅ All agents deleted and connections closed successfully!")



Then **uncomment the line below** and run it to cleanup

In [ ]:
#await cleanup_agents()

## 🎉 Congratulations!

You've built a multi-agent system with Azure AI Search!

### What You Accomplished
✅ Connected agents to Azure AI Search  
✅ Created specialized agents with different personalities  
✅ Built a router to send questions to the right agent  
✅ Streamed responses with search citations  
✅ Tested with real product queries  

### Key Concepts
- **AIProjectClient** - Manages your Azure AI project and connections
- **Agent Creation** - Use `create_agent()` with search tool configuration
- **ChatAgent Wrapper** - Provides streaming and citation collection
- **Tool Resources** - Configure which search index each agent uses
- **Instructions** - Shape agent behavior and personality
- **Router Pattern** - Automatically route questions to the best agent

### How the Router Works

When you ask a question:

1. **You ask** → "What interior paint do you recommend for my bedroom?"
2. **Router analyzes** → Sees keywords like "recommend" and "use for"
3. **Picks agent** → Routes to Customer Service Agent
4. **Agent searches** → Queries the `zava-products` index
5. **Gets results** → Azure AI Search returns matching products
6. **Agent responds** → Formats answer based on its personality
7. **You see** → Friendly response with product suggestions and citations

Both agents use the **same product database** but respond **differently** based on their instructions.

### Next Steps
- Modify agent instructions to change their behavior
- Try different search query types (semantic, vector, hybrid)
- Add more specialized agents (e.g., Returns Agent, Shipping Agent)
- Build a more sophisticated router with AI-based routing
- Add conversation memory so agents remember past questions

### Learn More
- [Agent Framework Docs](https://learn.microsoft.com/agent-framework/)
- [Azure AI Search Docs](https://learn.microsoft.com/azure/search/)
- [Azure AI Projects SDK](https://learn.microsoft.com/python/api/azure-ai-projects)

Happy building! 🚀